## Integração de dados

#### Masterlens, CASTLES

In [3]:
import pandas as pd
from astropy.coordinates import Angle

In [4]:
# dec: -27:52:25.7 ---> -27.873806 deg
def sexdeg_to_deg(coord):
    result = Angle(coord + " degrees").degree
    return result #*3600


In [5]:
# ra: 00h49m41.89s ---> 12.424542 deg
def hour_to_deg(coord):  # string coord
    result = Angle(coord).degree
    return result #*3600

In [6]:
# ra: 00:49:41.89 ---> 00h49m41.89s
def hour(coord):
    result = coord.replace(":", 'h', 1)
    result = result.replace(":", 'm', 1)
    result += 's'
    return result

In [21]:
def castles():
        # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')

    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles['id'] = castles.index  # from 0 to 99

    return castles

In [47]:
def masterlens():
        # importing masterlens data  -  50 lenses
    dfs = []
    for i in range(1,51):
        
        # getting data from system file
        ml = pd.read_csv('Scraping/results/system_{}.csv'.format(i))

        # selecting important columns
        cols=[]
        for column in ['Name', 'Alternate Names', 'Discovery', 'Discovery Date', 'Lens Kind', 'Lens Grade',
                       'Description', 'N Images', 'Einstein_R', 'Einstein_R quality',
                       'Stellar_v_disp', 'Stellar_v_disp_err']:
            if column in list(ml.columns):
                cols.append(column)
        ml = ml[cols]

        # getting coords from coords file
        coords = pd.read_csv('Scraping/results/coordinates_{}.csv'.format(i))
        ml['Ra(deg)_ml'] = coords['RA [°]']
        ml['Dec(deg)_ml'] = coords['Dec [°]']

        dfs.append(ml)
        
    all_ml = pd.concat(dfs, sort=False)
    all_ml['id'] = list(range(100, 150))  # from 100 to 149
    all_ml = all_ml.set_index(all_ml['id'])
        
    return all_ml

## Lens as a whole
### Merge  - Name

In [5]:
# system table ---------->  143 lenses
def system():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')
    castles.drop(['zs', 'zl', 'ms (mag)', 'ml (mag)', 'dt (days)'], axis=1, inplace=True)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles.columns=['Name', 'RA(deg)_c', 'Dec(deg)_c', 'E(B-V)_c', 'FGHz (mJy)_c', 'Nimages_c',
                     'size(arccsec)_c', 'sigma (km/s)_c']
    #castles = castles.set_index('Name')
    castles['id_c'] = castles.index
    
    
    # importing masterlens data  -  50 lenses
    dfs = []
    for i in range(1,51):
        
        # getting data from system file
        ml = pd.read_csv('Scraping/results/system_{}.csv'.format(i))

        # selecting important columns
        cols=[]
        for column in ['Name', 'Alternate Names', 'Discovery', 'Discovery Date', 'Lens Kind', 'Lens Grade',
                       'Description', 'N Images', 'Einstein_R', 'Einstein_R quality',
                       'Stellar_v_disp', 'Stellar_v_disp_err']:
            if column in list(ml.columns):
                cols.append(column)
        ml = ml[cols]

        # getting coords from coords file
        coords = pd.read_csv('Scraping/results/coordinates_{}.csv'.format(i))
        ml['Ra(deg)_ml'] = coords['RA [°]']
        ml['Dec(deg)_ml'] = coords['Dec [°]']

        dfs.append(ml)
     
    # joining all masterlens system + coords dfs
    all_ml = pd.concat(dfs, sort=False)
    #all_ml = all_ml.set_index('Name')
    #all_ml['id_ml'] = range(100, 150)
    
    
    concat = pd.concat([castles, all_ml], sort=False)
    concat.reset_index(inplace=True, drop=True)
    concat.to_csv('concat.csv')
    # joining castles + ml data
    result = pd.merge(castles, all_ml, how ='outer', on='Name')  


    return result
        


In [6]:
system_df = system()
system_df

,Name,RA(deg)_c,Dec(deg)_c,E(B-V)_c,FGHz (mJy)_c,Nimages_c,size(arccsec)_c,sigma (km/s)_c,id_c,Discovery,...,Lens Grade,Description,N Images,Einstein_R,Einstein_R quality,Stellar_v_disp,Stellar_v_disp_err,Ra(deg)_ml,Dec(deg)_ml,Alternate Names
0,Q0047-2808,12.424542,-27.873806,0.016,NaN,4ER,2.70,229±15,0.0,Serendipitous,...,A,Bright Einstein ring-like feature. The source ...,4.0,1.340,NaN,219.0,12.0,12.424747,-27.874020,ER 0047-2808
1,HE0047-1756,12.615958,-17.669111,0.022,NaN,2,1.44,NaN,1.0,HE survey,...,A,The newly discovered double QSO HE 0047−1756 ...,2.0,0.751,SIE model,NaN,NaN,12.615500,-17.668890,NaN
2,HST01247+0352,21.185000,3.866667,0.029,NaN,2,2.20,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HST01248+0351,21.190000,3.851667,0.029,NaN,2,0.74,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B0128+437,22.805854,43.970317,0.082,F5=48,4,0.55,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,SDSS J0956+5100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SLACS,...,A,This system includes a small round red compan...,3.0,1.330,SIE model,297.0,16.0,149.124060,51.001860,NaN
139,COSMOS5921+0638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSMOS,...,A,4-image + ring,4.0,0.620,SIE model,NaN,NaN,149.840680,2.110670,SL2SJ095921+020638
140,SDSS J0959+0410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SLACS,...,A,Multiply imaged distant bulge and grand spiral...,2.0,0.990,SIE model,203.0,12.0,149.933610,4.171390,Grand Design Source.
141,COSMOS0038+4133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSMOS,...,A,complete ring with brightness distribution con...,0.0,0.600,SIE model,NaN,NaN,150.159480,2.692735,NaN


### Merge - NACluster

In [7]:
# Preparing to run NACluster
def system_cluster():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')
    castles = castles[['RA (J2000)', 'Dec (J2000)' ]]
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles.columns=['ra', 'dec']  # degrees
    
    castles['idCatalog'] = 1
    #castles = castles.set_index('Name')
    
    
    # importing masterlens data  -  50 lenses
    dfs = []
    for i in range(1,51):

        # getting coords from coords file
        ml = pd.read_csv('Scraping/results/coordinates_{}.csv'.format(i))
        ml = ml[['RA [°]', 'Dec [°]']]
        ml.columns=['ra', 'dec']  # degrees
        ml['idCatalog'] = 2

        dfs.append(ml)
     
    # joining all masterlens system + coords dfs
    all_ml = pd.concat(dfs, sort=False)
    
    # concatenating both catalogs to use Nacluster
    final = pd.concat([castles,all_ml])
    final.reset_index(drop = True, inplace=True)
    final = final.sample(frac=1)  # shuffling dataframe
    
    # saving csv file
    #system_na .to_csv('catalogs.csv', header=False)

    return all_ml #final
        


In [8]:
system_na = system_cluster()

In [9]:
system_na

,ra,dec,idCatalog
0,2.012310,-0.068960,2
0,3.595917,-30.352722,2
0,3.609417,-30.382194,2
0,5.670492,14.519565,2
0,7.282400,-0.930710,2
0,9.271704,9.163992,2
0,11.012080,1.220180,2
0,12.424747,-27.874020,2
0,12.615500,-17.668890,2
0,17.390540,15.009030,2


In [58]:
# After running NACluster (Java)
# system table  ------------> 138 clustres
def NAC_result():
    old_file = open("clusters_catalogs.csv", 'r+')
    new_file = open("clusters.csv", 'a+')

    lastline=''
    cluster = 0
    header='id,ra,dec,cluster\n'
    new_file.write(header)
    for line in old_file:

        if 'cluster' in line:
            cluster+=1

        elif "Centroid" not in line:
            line = line.replace('(', '')
            line = line.replace(')', '')
            
            newline = ','.join(line.split(',')[0:3]) + ',{}'.format(cluster)
            new_file.write(newline)
            new_file.write('\n')
            
               
    old_file.close()
    new_file.close()
    
    return None
        

In [59]:
NAC_result()

In [66]:
nac = pd.read_csv('clusters.csv')
nac = nac.sort_values(by=['id'])
nac.reset_index(inplace=True, drop=True)
castle = castles()
ml = masterlens()

In [79]:
# first concat nac with castle
result = pd.concat([nac, castle], join ='outer', sort=True)  

# then concat nac + castle with ml
final_result = pd.concat([result, ml], join ='outer', sort=True) 


In [80]:
# combining dup rows into one
clusters = list(nac["cluster"])
dupes = list(set([x for x in clusters if clusters.count(x) > 1])) # cluster duplicates

for i in dupes:
    if final_result.index == i:
        print('bjhbj')
    

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [139]:
# Comparing merge by name with NAC

clusters = list(nac["cluster"])
dupes = list(set([x for x in clusters if clusters.count(x) > 1])) # cluster duplicates

clusters_dupes = nac.loc[nac['cluster'].isin(dupes)] 
#clusters_dupes

#merge_name = pd.read_csv('concat.csv')
#problem = [31, 141, 37, 149, 27, 135, 12, 125, 10, 122]
#merge_name.loc[merge_name.index.isin(problem)]

# Merge by name is wrong
# Use merge with NAC

## Lens objects

In [ ]:
# Scraping Joao
# lens objects  -  227
castles2 = pd.read_csv('CastelLensData.csv')
castles2 = castles2.set_index('lens_names')

HE_c = castles2.loc[ 'HE0435-1223' , : ]
HE_c

In [ ]:
# Object table
HE_ml = pd.read_csv('Scraping/results/redshift_28.csv')
HE_ml.drop('Unnamed: 1', axis=1, inplace=True)
HE_ml.rename(columns={"Unnamed: 0": "Description"})